# 네이버 블로그 발행 정보 최신순 수집 프로그램

## 프로그램 설명
네이버 블로그 검색 API를 사용하여 특정 키워드로 검색된 블로그 포스트를 최신순으로 수집하는 프로그램입니다.

## 주요 기능
- 네이버 블로그 검색 API 활용
- 최신순 블로그 포스트 수집
- 블로그 제목, 링크, 발행일 등 정보 수집
- 엑셀 파일 저장

## 사용 방법
1. 환경 변수에 NAVER_CLIENT_ID, NAVER_CLIENT_SECRET 설정
2. 프로그램 실행
3. 검색할 키워드 입력
4. 출력할 결과 건수 입력 (1~100)
5. 저장할 파일명 입력
6. 자동으로 블로그 정보 수집 및 엑셀 저장

## 필요 라이브러리
- requests: API 요청
- pandas: 데이터 처리
- openpyxl: 엑셀 파일 처리

## 주의사항
- 네이버 API 키가 필요합니다 (환경 변수로 설정)
- 출력 건수는 1~100 사이여야 합니다
- API 사용량 제한이 있을 수 있습니다

In [3]:
import requests
import pandas as pd
import os
from datetime import datetime

# 환경 변수에서 네이버 API 정보 가져오기
client_id = os.getenv('NAVER_CLIENT_ID')
client_secret = os.getenv('NAVER_CLIENT_SECRET')

if not client_id or not client_secret:
    raise ValueError("NAVER_CLIENT_ID와 NAVER_CLIENT_SECRET 환경 변수를 설정해주세요.")

# 검색할 키워드와 출력 건수, 저장 파일명 입력 받기
keyword = input("검색할 키워드를 입력하세요: ")
display_count = input("출력할 검색 결과의 건수를 입력하세요 (1~100): ")
file_name = input("저장할 파일명을 입력하세요 (확장자 제외): ")

# 입력된 출력 건수 유효성 검사
try:
    display_count = int(display_count)
    if display_count < 1 or display_count > 100:
        raise ValueError
except ValueError:
    raise ValueError("출력 건수는 1에서 100 사이의 정수여야 합니다.")

# 네이버 블로그 검색 API URL
url = 'https://openapi.naver.com/v1/search/blog.json'

# 요청 헤더
headers = {
    'X-Naver-Client-Id': client_id,
    'X-Naver-Client-Secret': client_secret
}

# 요청 파라미터
params = {
    'query': keyword,
    'display': display_count,  # 검색 결과의 출력 건수
    'start': 1,                # 검색 시작 위치
    'sort': 'date'             # 정렬 옵션: date(날짜순), sim(유사도순)
}

# API 요청
response = requests.get(url, headers=headers, params=params)

# 응답 데이터 확인
if response.status_code == 200:
    result = response.json()
    items = result['items']
    
    # 결과를 저장할 리스트
    data = []
    
    for idx, item in enumerate(items, start=1):
        title = item['title']
        link = item['link']
        description = item['description']
        bloggername = item['bloggername']
        postdate = item['postdate']
        
        data.append({
            '순번': idx,
            '제목': title,
            'URL': link,
            '요약': description,
            '블로거 이름': bloggername,
            '포스트 날짜': postdate
        })
        
        # 화면에 출력
        print(f'순번: {idx}')
        print(f'제목: {title}')
        print(f'URL: {link}')
        print(f'요약: {description}')
        print(f'블로거 이름: {bloggername}')
        print(f'포스트 날짜: {postdate}')
        print('------')
    
    # 데이터프레임으로 변환
    df = pd.DataFrame(data)
    
    # 오늘 날짜를 yyyymmdd 형식으로 얻기
    today = datetime.today().strftime('%Y%m%d')
    
    # 엑셀 파일로 저장 (파일 이름에 경로 추가 및 날짜 접미사 추가)
    output_directory = os.path.expanduser("~")
    excel_filename = os.path.join(output_directory, f'{file_name}_{today}.xlsx')
    df.to_excel(excel_filename, index=False)
    
    print(f'검색 결과가 엑셀 파일로 저장되었습니다: {excel_filename}')
    
    # 엑셀 파일 열기
    os.system(f'start excel "{excel_filename}"')
else:
    print('API 요청에 실패했습니다. 상태 코드:', response.status_code)
    print('오류 메시지:', response.json())


검색할 키워드를 입력하세요: 니뜨
출력할 검색 결과의 건수를 입력하세요 (1~100): 100
저장할 파일명을 입력하세요 (확장자 제외): 니뜨블로그발행
순번: 1
제목: 생생한 목얼음주머니 경험 여름 체온을 낮춰주는 쿨링 아이템
URL: https://jkh3460.tistory.com/969
요약: 다<b>니뜨</b> 얼음 주머니 휴대용 아이스 팩 골프 냉 온 찜질 1개, K2 1 1 아이스글랜 쿨스카프 얼음스카프 그레이 그레이, 인기몰 1 1 아메바 냉동 스카프 남녀공용면 아사 스카프 1 1아메바_네이비, 코쿠보 열... 
블로거 이름: jkh3460
포스트 날짜: 20240602
------
순번: 2
제목: 추천! 화이트 양털조끼, 품절임박! 놀라운 털질과 보온력 - 상세... 
URL: https://blog.naver.com/kpf5758q/223466637748
요약: 인기 후기 Top3 위 ■ 다<b>니뜨</b> 여성 양털조끼 뽀글이 집업 후리스 데일리 레이어드 빅사이즈 베스트 DA-338  상품가격  ↓ 38,000원 ↓ 61% 할인 ↓ 14,610원 로켓배송 (유/무) ❌ (⭐별점⭐ 총 4.5점... 
블로거 이름: kpf5758q님의 블로그
포스트 날짜: 20240602
------
순번: 3
제목: [부산] 감성 소품 잔뜩 파는 전포 소품샵&amp;편집샵 {띵스오브노트}
URL: https://blog.naver.com/peanut6utterr/223466433662
요약: #그리<b>니뜨</b> 여름에 포인트로 들기 좋을 거 같아서 눈길이 갔답니당 한쪽 구석에는 엽서랑 포스터들도 잔뜩 있었어요~ 귀여운 그림도 많고 감성 넘치는 사진도 많아서 생일 편지 쓰셔야 하는 분들... 
블로거 이름: Love is all
포스트 날짜: 20240602
------
순번: 4
제목: 세면대수전으로 물보다 깊은 청결함을 누려보세요
URL: https://pil97.tistory.com/215
요약: 다<b>니뜨</b> 워터탭 듀얼분사 1440도 각도